In [1]:
import pandas as pd
import numpy as np
import os
import psycopg2

In [2]:
# Connect to your postgres DB
conn = psycopg2.connect(dbname='Yellow-Taxi', user='postgres', password='7466239', host='localhost', port='5432')

# Open a cursor to perform database operations
cur = conn.cursor()

## table for yellow taxi data

### i will insert the data into taxi_data table using the copy method directly in Pgadmin

In [19]:
# Create table in PostgreSQL
cur.execute('''
    CREATE TABLE IF NOT EXISTS taxi_data (
        VendorID INT,
        tpep_pickup_datetime TIMESTAMP,
        tpep_dropoff_datetime TIMESTAMP,
        passenger_count INT,
        trip_distance FLOAT,
        RatecodeID INT,
        store_and_fwd_flag CHAR(1),
        PULocationID INT,
        DOLocationID INT,
        payment_type INT,
        fare_amount FLOAT,
        extra FLOAT,
        mta_tax FLOAT,
        tip_amount FLOAT,
        tolls_amount FLOAT,
        improvement_surcharge FLOAT,
        total_amount FLOAT,
        congestion_surcharge FLOAT,
        airport_fee FLOAT
    )
''')
conn.commit()

## Climate data table

In [7]:
cur.execute('''
CREATE TABLE IF NOT EXISTS climate (
    station_id varchar,
    station_name varchar,
    date timestamp,
    latitude float,
    longitude float,
    elevation float,
    temperature float,
    dew_point_temperature float,
    station_level_pressure float,
    sea_level_pressure float,
    wind_direction float,
    wind_speed float,
    wind_gust float,
    precipitation float,
    relative_humidity float,
    wet_bulb_temperature float,
    pres_wx_mw1 varchar,
    pres_wx_mw2 varchar,
    pres_wx_mw3 varchar,
    pres_wx_au1 varchar,
    pres_wx_au2 varchar,
    pres_wx_au3 varchar,
    pres_wx_aw1 varchar,
    pres_wx_aw2 varchar,
    pres_wx_aw3 varchar,
    snow_depth float,
    visibility float,
    altimeter float,
    pressure_3hr_change float,
    sky_cover_1 varchar,
    sky_cover_baseht_1 float,
    sky_cover_2 varchar,
    sky_cover_baseht_2 float,
    sky_cover_3 varchar,
    sky_cover_baseht_3 float,
    precipitation_3_hour float,
    precipitation_6_hour float,
    precipitation_9_hour float,
    precipitation_12_hour float,
    precipitation_15_hour float,
    precipitation_18_hour float,
    precipitation_21_hour float,
    precipitation_24_hour float,
    remarks varchar
    )
''')
conn.commit()

In [6]:
cur.execute('''drop table if exists climate''')
conn.commit()

In [3]:
# Read data from PostgreSQL table into a pandas DataFrame
pd.read_sql_query('SELECT * FROM taxi_data limit 3', conn)

C:\Users\Mark Rozenberg\AppData\Local\Temp\ipykernel_16084\1481243530.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql_query('SELECT * FROM taxi_data limit 3', conn)


,vendorid,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,ratecodeid,store_and_fwd_flag,pulocationid,dolocationid,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
0,2,2023-01-01 00:32:10,2023-01-01 00:40:36,1,0.97,1,N,161,141,2,9.3,1.0,0.5,0.0,0.0,1.0,14.3,2.5,0.0
1,2,2023-01-01 00:55:08,2023-01-01 01:01:27,1,1.10,1,N,43,237,1,7.9,1.0,0.5,4.0,0.0,1.0,16.9,2.5,0.0
2,2,2023-01-01 00:25:04,2023-01-01 00:37:49,1,2.51,1,N,48,238,1,14.9,1.0,0.5,15.0,0.0,1.0,34.9,2.5,0.0


### create combined table that hold taxi data with climate data

In [15]:
pd.read_sql_query('''
select CASE 
    WHEN EXTRACT(minute FROM tpep_pickup_datetime) > 30 THEN date_trunc('hour', tpep_pickup_datetime + interval '1 hour')
    ELSE date_trunc('hour', tpep_pickup_datetime)
END AS rounded_date, *
from taxi_data
limit 3
''', conn)

C:\Users\Mark Rozenberg\AppData\Local\Temp\ipykernel_11524\1076832850.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql_query('''


,rounded_date,vendorid,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,ratecodeid,store_and_fwd_flag,pulocationid,dolocationid,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
0,2023-01-01 01:00:00,2,2023-01-01 00:32:10,2023-01-01 00:40:36,1,0.97,1,N,161,141,2,9.3,1.0,0.5,0.0,0.0,1.0,14.3,2.5,0.0
1,2023-01-01 01:00:00,2,2023-01-01 00:55:08,2023-01-01 01:01:27,1,1.10,1,N,43,237,1,7.9,1.0,0.5,4.0,0.0,1.0,16.9,2.5,0.0
2,2023-01-01 00:00:00,2,2023-01-01 00:25:04,2023-01-01 00:37:49,1,2.51,1,N,48,238,1,14.9,1.0,0.5,15.0,0.0,1.0,34.9,2.5,0.0


In [5]:
pd.read_sql_query('''
select * from climate limit 3
''', conn)

C:\Users\Mark Rozenberg\AppData\Local\Temp\ipykernel_21456\2394927617.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql_query('''


,station_id,station_name,date,latitude,longitude,elevation,temperature,dew_point_temperature,station_level_pressure,sea_level_pressure,...,sky_cover_baseht_3,precipitation_3_hour,precipitation_6_hour,precipitation_9_hour,precipitation_12_hour,precipitation_15_hour,precipitation_18_hour,precipitation_21_hour,precipitation_24_hour,remarks
0,USI0000KJRB,PORT AUTH DOWNTN MANHATTAN WAL,2022-02-01 16:56:00,40.7012,-74.009,2.1,2.2,-5.6,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,MET085METAR KJRB 011656Z AUTO 06009KT 10SM CLR...
1,USI0000KJRB,PORT AUTH DOWNTN MANHATTAN WAL,2022-02-01 17:56:00,40.7012,-74.009,2.1,3.3,-5.6,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,MET084METAR KJRB 011756Z AUTO 05007KT 10SM CLR...
2,USI0000KJRB,PORT AUTH DOWNTN MANHATTAN WAL,2022-02-01 18:56:00,40.7012,-74.009,2.1,3.9,-5.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,MET078METAR KJRB 011856Z AUTO 05008KT 10SM CLR...


In [3]:
pd.read_sql_query('''
select * from taxi_data limit 3
''', conn)

C:\Users\Mark Rozenberg\AppData\Local\Temp\ipykernel_21456\1920312658.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql_query('''


,vendorid,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,ratecodeid,store_and_fwd_flag,pulocationid,dolocationid,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee,rounded_date
0,2,2023-05-31 23:00:07,2023-05-31 23:42:35,None,11.44,None,None,71,246,0,49.33,0.0,0.5,0.00,6.55,1.0,59.88,None,None,2023-05-31 23:00:00
1,2,2023-08-01 00:20:26,2023-08-01 00:42:22,None,6.90,None,None,230,244,0,24.89,0.0,0.5,0.00,0.00,1.0,28.89,None,None,2023-08-01 00:00:00
2,2,2022-02-28 23:38:52,2022-03-01 00:09:41,None,11.32,None,None,142,89,0,36.98,0.0,0.5,10.21,6.55,0.3,57.04,None,None,2022-03-01 00:00:00


In [47]:
# Add rounded_date column to taxi_data table
cur.execute('''
ALTER TABLE taxi_data
ADD COLUMN rounded_date TIMESTAMP
''')
conn.commit()

# Update the rounded_date column with the calculated values
cur.execute('''
UPDATE taxi_data
SET rounded_date = CASE 
    WHEN EXTRACT(minute FROM tpep_pickup_datetime) > 30 THEN date_trunc('hour', tpep_pickup_datetime + interval '1 hour')
    ELSE date_trunc('hour', tpep_pickup_datetime)
END
''')
conn.commit()

In [6]:
# Add rounded_date column to climate table
cur.execute('''
ALTER TABLE climate
ADD COLUMN rounded_date TIMESTAMP
''')
conn.commit()

# Update the rounded_date column with the calculated values
cur.execute('''
UPDATE climate
SET rounded_date = CASE 
    WHEN EXTRACT(minute FROM date) > 30 THEN date_trunc('hour', date + interval '1 hour')
    ELSE date_trunc('hour', date)
END
''')
conn.commit()

In [4]:
# Add new columns to taxi_data table
cur.execute('''
ALTER TABLE taxi_data
ADD COLUMN avg_temperature FLOAT,
ADD COLUMN avg_wind_speed FLOAT,
ADD COLUMN avg_precipitation FLOAT,
ADD COLUMN avg_relative_humidity FLOAT,
ADD COLUMN avg_visibility FLOAT
''')
conn.commit()

In [ ]:
cur.execute(''' ALTER TABLE taxi_data DROP COLUMN avg_temperature ''')
conn.commit()

In [5]:
cur.execute(''' ALTER TABLE taxi_data DROP COLUMN avg_temperature ''')
conn.commit()
cur.execute(''' ALTER TABLE taxi_data DROP COLUMN avg_wind_speed ''')
cur.execute(''' ALTER TABLE taxi_data DROP COLUMN avg_precipitation ''')
cur.execute(''' ALTER TABLE taxi_data DROP COLUMN avg_relative_humidity ''')
cur.execute(''' ALTER TABLE taxi_data DROP COLUMN avg_visibility ''')
conn.commit()

InFailedSqlTransaction: current transaction is aborted, commands ignored until end of transaction block


In [10]:
# Update taxi_data table with the calculated values
cur.execute('''
WITH climate_data AS (
    SELECT 
        rounded_date,
        avg(temperature) AS avg_temperature,
        avg(wind_speed) AS avg_wind_speed,
        avg(precipitation) AS avg_precipitation,
        avg(relative_humidity) AS avg_relative_humidity,
        avg(visibility) AS avg_visibility
    FROM climate
    WHERE temperature != 'NaN' AND wind_speed != 'NaN' AND relative_humidity != 'NaN' AND visibility != 'NaN'
    GROUP BY 1
)
UPDATE taxi_data
SET 
    avg_temperature = climate_data.avg_temperature,
    avg_wind_speed = climate_data.avg_wind_speed,
    avg_precipitation = climate_data.avg_precipitation,
    avg_relative_humidity = climate_data.avg_relative_humidity,
    avg_visibility = climate_data.avg_visibility
FROM climate_data
WHERE 
    taxi_data.rounded_date = climate_data.rounded_date
''')
conn.commit()

In [ ]:
pd.read_sql_query('''
select wind_speed, count(*)
from climate
group by 1
order by 2 desc
''', conn)

In [ ]:
pd.read_sql_query('''
select count(*)
from climate
where wind_speed is not null
''', conn)

C:\Users\Mark Rozenberg\AppData\Local\Temp\ipykernel_11524\1200688290.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql_query('''


,count
0,250414


In [39]:
pd.read_sql_query(''' select wind_speed from climate limit 10''', conn).dtypes

C:\Users\Mark Rozenberg\AppData\Local\Temp\ipykernel_11524\2948983072.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql_query(''' select wind_speed from climate limit 10''', conn).dtypes


wind_speed    float64
dtype: object

In [38]:
pd.read_sql_query('''
select count(*)
from climate
where wind_speed = 'NaN'
''', conn)

C:\Users\Mark Rozenberg\AppData\Local\Temp\ipykernel_11524\1081546797.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql_query('''


,count
0,16810
